<a href="https://colab.research.google.com/github/gkv856/KaggleData/blob/main/GE2EL_v001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from tensorflow.keras.layers import LSTM, Input
from tensorflow import Variable
import tensorflow.keras.layers as layers

In [2]:
!git clone https://github.com/gkv856/gkv856-GE2EL_tensorflow_implementation_with_data.git

Cloning into 'gkv856-GE2EL_tensorflow_implementation_with_data'...
remote: Enumerating objects: 429, done.
remote: Counting objects: 100% (429/429), done.
remote: Compressing objects: 100% (419/419), done.
remote: Total 429 (delta 7), reused 428 (delta 6), pack-reused 0
Receiving objects: 100% (429/429), 87.47 MiB | 24.42 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [4]:
from gkv.string_constants.configuration_file import *
from gkv.gkv_implement.util import *

In [86]:
config["train_path"] = "/content/gkv/audio_data/spectrograms/train"
config["embed_dim"] = 128
config["lr"] = 1e-6

# config

In [87]:
def create_model(inp_shape, m_name="model"):

    # input batch (time x batch x n_mel)
    inputs = layers.Input(shape=inp_shape, dtype=tf.float32)
    x = inputs
    x = layers.LSTM(units=config["embed_dim"], return_sequences=True, name="l1")(x)
    x = layers.LSTM(units=config["embed_dim"], return_sequences=True, name="l2")(x)
    x = layers.LSTM(units=config["n_mels"], dtype=tf.float32, return_sequences=True, return_state=True, name="l3")(x)
    # embedded = x[-1]
    # norm_embedding = normalize(x)
    # for y in x:
    #   print("norm_embedding embedded size: ", y)

    model = tf.keras.Model(inputs, outputs=x, name=m_name)
    return model


In [88]:
# len_utterence = None # since each batch will have different length
inp_shape = [config["speaker_num"]*config["utter_num"], config["n_mels"]]
model = create_model(inp_shape, "model0")
# model.summary()

In [89]:
model_optimizer = get_optimizer(lr=config["lr"])
from gkv.gkv_implement.util import *

In [90]:
# @tf.function
def fit_train_model(model, epochs=10):
    # tf.reset_default_graph()
    center = None
    w = Variable(10, name="w", dtype=tf.float32)
    b = Variable(-5, name="b", dtype=tf.float32)

    for epoch in range(epochs):
      inputs = random_batch()
      # print(f"Input shape = {inputs.shape}")
      
      with tf.GradientTape(persistent=False) as tape:
          model_outputs = model(inputs)
          # print(f"model output shape = {model_outputs[0].shape}")
          out = model_outputs[0]
          # print(out.shape, type(out))
          # embedded = model_outputs[-1]  # the last ouput is the embedded d-vector
          # norm_embedded = normalize(embedded)
          N = out.shape[1]
          M = out.shape[2]
          P = out.shape[0]
          # reshaped_lstm_embedding = tf.reshape(out, shape=[out.shape[1], out.shape[2], out.shape[0]])

          reshaped_lstm_embedding = tf.reshape(out, shape=[N, M, P])
          # print(reshaped_lstm_embedding.shape)

          embedded_split = reshaped_lstm_embedding
          # print(f"Center = {center}")
          if True:#center is None:
            center = normalize(tf.reduce_mean(embedded_split, axis=1))              # [N,P] normalized center vectors eq.(1)
            center_except = normalize(tf.reshape(tf.reduce_sum(embedded_split, axis=1, keepdims=True)
                                                - embedded_split, shape=[N*M,P]))  # [NM,P] center vectors eq.(8)
            # make similarity matrix eq.(9)
            S = tf.concat(
                [tf.concat([tf.reduce_sum(center_except[i*M:(i+1)*M,:]*embedded_split[j,:,:], axis=1, keepdims=True) if i==j
                            else tf.reduce_sum(center[i:(i+1),:]*embedded_split[j,:,:], axis=1, keepdims=True) for i in range(N)],
                          axis=1) for j in range(N)], axis=0)
            # print(f"1 S = {S.shape}")
          # else :
          #   # If center(enrollment) exist, use it.
          #   S = tf.concat(
          #       [tf.concat([tf.reduce_sum(center[i:(i + 1), :] * embedded_split[j, :, :], axis=1, keepdims=True) for i
          #                   in range(N)],
          #                 axis=1) for j in range(N)], axis=0)
          #   print(f"2 S = {S.shape}")

          S = tf.abs(w)*S+b   # rescaling
          sim_matrix = S


          # sim_matrix = calc_similarity(out, w, b, N, M, P)
          # print("similarity matrix size: ", sim_matrix.shape)

          S_correct = tf.concat([S[i * M:(i + 1) * M, i:(i + 1)] for i in range(N)], axis=0)
          # print(f"S_correct = {S_correct.shape}")
          
          tf_log = tf.math.log(tf.reduce_sum(tf.exp(S), axis=1, keepdims=True) + 1e-6)
          # print(f"tf log = {tf_log.shape}")
          tf_correct = S_correct - tf_log
          total = -tf.reduce_sum(tf_correct)

          loss = total #calculate_loss(sim_matrix, type=config["loss"])

          m_gradients = tape.gradient(loss, model.trainable_variables)

          model_optimizer.apply_gradients(zip(m_gradients, model.trainable_variables))
            
      print(f'Epoch{epoch} - loss = {loss}...')


In [91]:
fit_train_model(model, epochs=100)

Epoch0 - loss = 4201.73193359375...
Epoch1 - loss = 2998.325927734375...
Epoch2 - loss = 2500.6650390625...
Epoch3 - loss = 2847.919677734375...
Epoch4 - loss = 2867.009765625...
Epoch5 - loss = 2819.614990234375...
Epoch6 - loss = 2748.192626953125...
Epoch7 - loss = 2638.098876953125...
Epoch8 - loss = 2721.247314453125...
Epoch9 - loss = 2725.575439453125...
Epoch10 - loss = 2633.799560546875...
Epoch11 - loss = 2681.768310546875...
Epoch12 - loss = 2648.532958984375...
Epoch13 - loss = 2597.80224609375...
Epoch14 - loss = 2593.49462890625...
Epoch15 - loss = 2468.27880859375...
Epoch16 - loss = 2609.0654296875...
Epoch17 - loss = 2574.455078125...
Epoch18 - loss = 2588.8623046875...
Epoch19 - loss = 2556.059814453125...
Epoch20 - loss = 2559.06103515625...
Epoch21 - loss = 2428.72509765625...
Epoch22 - loss = 2551.76123046875...
Epoch23 - loss = 2526.975341796875...
Epoch24 - loss = 2472.829345703125...
Epoch25 - loss = 2539.287841796875...
Epoch26 - loss = 2529.3779296875...
Epoch

In [ ]:
# (20, 40, 174)
# Center = None
# 1 S = (800, 20)
# similarity matrix size:  (800, 20)
# S_correct = (800, 1)
# tf log = (800, 1)

# 1 S = (800, 20)
# similarity matrix size:  (800, 20)
# S_correct = (800, 1)
# tf log = (800, 1)

# Epoch0 - loss = 3105.39892578125...

# Epoch1 - loss = 2931.333984375...